#Viewing Changes in the Delta Log files to Extract Schema Evolution





Delta log files show version history of our delta tables. 

Basically we can read the *.json delta log files for each iteration in order to create reporting on the schema. The log files are dictionaries with nested jsons within them so it's a little difficult to get at the data we want without understanding Struct data types.

Struct data types can be exploded or accessed using a database like dot method colname.subcolname

First we will see the log files we have in memory

In [0]:
%fs ls dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log

path,name,size,modificationTime
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000000.crc,00000000000000000000.crc,2067,1724867480000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000000.json,00000000000000000000.json,1012,1724867464000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000001.00000000000000000006.compacted.json,00000000000000000001.00000000000000000006.compacted.json,2490,1724867569000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000001.crc,00000000000000000001.crc,2526,1724867501000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000001.json,00000000000000000001.json,1329,1724867499000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000002.crc,00000000000000000002.crc,3355,1724867511000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000002.json,00000000000000000002.json,1506,1724867507000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000003.crc,00000000000000000003.crc,2697,1724867539000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000003.json,00000000000000000003.json,1404,1724867538000
dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000004.crc,00000000000000000004.crc,4201,1724867543000


Viewing the raw file

In [0]:
jsonmulti = spark.read.option("multiline", "true").json("/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000009.json")

json1 = spark.read.json("/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/00000000000000000006.json")

display(json1)


commitInfo,metaData
"List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(-1, Append, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 5, List(false), 1724867567555, 4dd5fb59-db91-42f5-91ce-1e3e23380340, 7971536220129282, matthew.resner@group1001.com)",null
null,"List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]})"


From a file control and monitoring standpoint we might want to have the commitInfo Timestamp or metaData jsons which could be unpacked and organized to enable easy visualization and reporting on schema change for a particular delta table. 

To do that we want to get the commitInfo and metaData on the same lines so that we can extract timestamp from commitInfo and our schema evolution from metaData

In [0]:
#Joining the delta log to itself to get metadata and commitInfo on the same lines for each schema version

from pyspark.sql.functions import input_file_name, regexp_extract

spark.read.json("dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/*.json")\
    .withColumn("table_name", regexp_extract(input_file_name(), r'/tables/TraderJoesRevenue/(.*?)/_delta_log/', 1))\
    .withColumn("schema_version", regexp_extract(input_file_name(), r'(\d+)\.json$', 0))\
    .createOrReplaceTempView("delta_log")

display(spark.sql(" \
    select a.table_name, a.schema_version, a.metaData, b.commitInfo \
    from delta_log a \
    left join delta_log b on a.schema_version = b.schema_version AND b.commitInfo is not null\
    WHERE a.metaData is not null and a.schema_version <> '' \
    ORDER BY a.schema_version DESC"))

table_name,schema_version,metaData,commitInfo
bronze_tj_fact_revenue,00000000000000000006.json,"List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]})","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 5, List(false), 1724867567555, 4dd5fb59-db91-42f5-91ce-1e3e23380340, 7971536220129282, matthew.resner@group1001.com)"
bronze_tj_fact_revenue,00000000000000000003.json,"List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}}]})","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 2, List(false), 1724867538429, fccaa851-0892-4742-b042-c680336e99a2, 7971536220129282, matthew.resner@group1001.com)"
bronze_tj_fact_revenue,00000000000000000001.json,"List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}}]})","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 0, List(false), 1724867499107, a5a5c724-5609-4ba2-bef3-9de94d73748a, 7971536220129282, matthew.resner@group1001.com)"
bronze_tj_fact_revenue,00000000000000000000.json,"List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[]})","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(285342671664234), CREATE TABLE, List(null, null, null, null), List([], null, null, true, null, [], {""delta.enableDeletionVectors"":""true""}, null, false), null, List(false), 1724867463892, 74052df9-40b0-48ee-b209-10e161b89246, 7971536220129282, matthew.resner@group1001.com)"


Next we add a window function (LAG) in order to get the previous schema on the same line with the current schema (then we will compare them)

In [0]:
# Struct Selection to access schema variances using windowing
from pyspark.sql.functions import input_file_name, regexp_extract, col, explode, struct, from_unixtime

spark.read.json("dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue/_delta_log/*.json")\
    .withColumn("table_name", regexp_extract(input_file_name(), r'/tables/TraderJoesRevenue/(.*?)/_delta_log/', 1))\
    .withColumn("schema_version", regexp_extract(input_file_name(), r'(\d+)\.json$', 0))\
    .createOrReplaceTempView("delta_log")

#windowing function here
df =(spark.sql(" \
    select a.table_name, a.schema_version, from_unixtime(b.commitInfo.timestamp / 1000) as timestamp, a.metaData.schemaString AS current_schemaString, LAG(a.metaData.schemaString, 1) OVER(PARTITION BY a.table_name ORDER BY a.schema_version) AS prev_schemaString ,b.commitInfo , a.metaData  \
    from delta_log a \
    left join delta_log b on a.schema_version = b.schema_version AND b.commitInfo is not null\
    WHERE a.metaData is not null and a.schema_version <> '' \
    ORDER BY a.schema_version DESC"))

display(df)

table_name,schema_version,timestamp,current_schemaString,prev_schemaString,commitInfo,metaData
bronze_tj_fact_revenue,00000000000000000006.json,2024-08-28 17:52:47,"{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 5, List(false), 1724867567555, 4dd5fb59-db91-42f5-91ce-1e3e23380340, 7971536220129282, matthew.resner@group1001.com)","List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]})"
bronze_tj_fact_revenue,00000000000000000003.json,2024-08-28 17:52:18,"{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}}]}","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 2, List(false), 17248675

Now time for a UDF to compare the previous and current schema, find the differences and then write those differences to a new column using df.withcolumn again

In [0]:
from pyspark.sql.functions import col, from_json, udf
from pyspark.sql.types import StringType

display(df)

# write a UDF
def compare_schemas(current_schema, prev_schema):
    if prev_schema is None:
        return 'No previous schema'
        # Directly work with StructType's fields in two set operations to compare the two schemas current_schema.fields and prev_schema.fields
    current_fields = set(field.name for field in current_schema.fields) if current_schema else set() 
        
    prev_fields = set(field.name for field in prev_schema.fields) if prev_schema else set() 
        
    added = current_fields - prev_fields
    removed = prev_fields - current_fields
    return f"Added: {list(added)}, Removed: {list(removed)}"

# Converting the output to stringtype (the code doesn't work without this which I don't completly understand TBH)
compare_schemas_udf = udf(compare_schemas, StringType())

# Apply UDF to the Date Frame earlier
df = df.withColumn("schema_changes", compare_schemas_udf(from_json(col("current_schemaString"), schema), from_json(col("prev_schemaString"), schema)))

# Write a temp view again
df.createOrReplaceTempView("delta_log_enhanced")

# Display the result and reorder columns with SparkSQL
display(spark.sql("""
    SELECT table_name, schema_version, timestamp, schema_changes, current_schemaString, prev_schemaString, metaData, commitInfo
    FROM delta_log_enhanced
    ORDER BY schema_version DESC
"""))

table_name,schema_version,timestamp,current_schemaString,prev_schemaString,commitInfo,metaData,schema_changes
bronze_tj_fact_revenue,00000000000000000006.json,2024-08-28 17:52:47,"{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 5, List(false), 1724867567555, 4dd5fb59-db91-42f5-91ce-1e3e23380340, 7971536220129282, matthew.resner@group1001.com)","List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]})","Added: ['Profit'], Removed: []"
bronze_tj_fact_revenue,00000000000000000003.json,2024-08-28 17:52:18,"{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}}]}","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93e

table_name,schema_version,timestamp,schema_changes,current_schemaString,prev_schemaString,metaData,commitInfo
bronze_tj_fact_revenue,00000000000000000006.json,2024-08-28 17:52:47,"Added: ['Profit'], Removed: []","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Profit"",""type"":""double"",""nullable"":true,""metadata"":{}}]})","List(0801-194747-f0qk6m1o, Databricks-Runtime/15.3.x-cpu-ml-scala2.12, true, WriteSerializable, List(3309121777485539), STREAMING UPDATE, List(null, null, null, null), List(null, null, -1, null, Append, null, null, 58c82e50-133e-4f0e-93ed-ca0e65758278, false), 5, List(false), 1724867567555, 4dd5fb59-db91-42f5-91ce-1e3e23380340, 7971536220129282, matthew.resner@group1001.com)"
bronze_tj_fact_revenue,00000000000000000003.json,2024-08-28 17:52:18,"Added: ['COGS'], Removed: []","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""COGS"",""type"":""integer"",""nullable"":true,""metadata"":{}}]}","{""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Store_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Year"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""name"":""Quarter"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""Total_Revenue"",""type"":""string"",""nullable"":true,""metadata"":{}},{""name"":""_rescued_data"",""type"":""string"",""nullable"":true,""metadata"":{}}]}","List(List(true), 1724867463455, List(parquet), 0bde0511-7d01-4e42-a5fb-35d8b714d209, List(), {""type"":""struct"",""fields"":[{""name"":""Row_ID"",""type"":""integer"",""nullable"":true,""metadata"":{}},{""